# Langchain Demo with telegram data

In [2]:
import os
import pinecone
import pandas as pd
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv()) # read local .env file

from langchain.llms import OpenAIChat
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma
from chromadb.config import Settings

## Data loading

In [3]:
# load telegram messages
channel_id = "singularitynet"
df = pd.read_csv(f"data/{channel_id}.csv")
              
# Use langchain wrapper to load dataframe              
df_loader = DataFrameLoader(df, page_content_column="message")
docs = df_loader.load()

In [4]:
df["message"].iloc[0:4]

0    ⚠️ Safety Warning ‼️ \n\n‼️ The ASI token is n...
1    ⚠️WARNING!!!⚠️ 🚨SAFETY ALERT!!!🚨\n\nNeither Si...
2                                              !safety
3    We always include a warning, see last sentence...
Name: message, dtype: object

## Using OpenAI embeddings

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DataFrameLoader
from IPython.display import display, Markdown
import pandas as pd

In [10]:
channel_id = "singularitynet"
df = pd.read_csv(f"data/{channel_id}.csv")
df_loader = DataFrameLoader(df, page_content_column="message")
docs = df_loader.load()

In [11]:
import chromadb

persistent_client = chromadb.PersistentClient()

CLEAR_COLLECTION = True

if CLEAR_COLLECTION:
    try:
        persistent_client.delete_collection(f"openai_embeddings_{channel_id[1:]}")
    except Exception as e:
        print("unable to delete ", e)

# How to create a client with reset allowed
# client = chromadb.HttpClient(settings=Settings(allow_reset=True))
# client.reset()  # resets the database

unable to delete  Collection openai_embeddings_ingularitynet does not exist.


With our loaded collection of telegram messages, it is time to create embeddings using openAI. Chroma db offers us a simple way to achieve this.

Beware! This code performs several request to the openai API in order to create embeddings for each of each message in `docs`. Depending on the size of your dataset, this could incurre in high costs.

In [12]:
openai_embeddings_function = OpenAIEmbeddings()

openai_chroma_client = Chroma.from_documents(
    docs, 
    openai_embeddings_function, 
    client=persistent_client, 
    collection_name = f"openai_embeddings_{channel_id[1:]}"
)

/Users/camiloramirezf/Library/Caches/pypoetry/virtualenvs/telegram-nlp-cUHCjj8z-py3.10/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [14]:
# helper function to perform similarity search
def search_db(db_client, query: str, top_k = 100):
    docs = db_client.similarity_search(query, k=top_k)

    # print results
    for item in docs[:5]:
        print(f"\n{item.page_content}")

The embeddings are saved locally. Now we can perform search queries by obtaining the most similar documents to our query.

In [42]:
query = "use cases"
search_db(openai_chroma_client, query)

- in addition to securing the network you also provide a real world case for solving problems  eg genome sequencing, graphics rendering, etc...
- upto you to find and research
- Can you please write in simple words the usecases of flux coin ? Services which requires flux token ?
- What are the $FLUX usecases?Let's find out httpstwitter.comHouseofChimerastatus1671902756808818690
- It may help to 
- Harness the power of decentralisation Explore RunOnFlux use cases Let's read onhttpstwitter.comHouseofChimerastatus1709901477689364513
- cycle through the different options and see what the issue could be
- Usage is one of the key things that separates Flux from its competitors, show the below some love on twitter and help spread the word httpstwitter.comcryptoviumstatus1686329414940823552?s20
- Anyone can apply to use their services for payments
- bring the network to life


In [17]:
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(), 
    chain_type="stuff", 
    retriever=openai_chroma_client.as_retriever(search_type="mmr", search_kwargs={'fetch_k': 30}), 
    return_source_documents=True
)

In [18]:
query = "What are some use cases of flux?"
response = qa({"query":query})

In [19]:
response["result"]

"Some potential use cases for Flux could include:\n\n1. Real-time data processing and analytics: Flux is designed to handle large volumes of data and perform computations in real-time, making it suitable for applications that require fast data processing and analytics.\n\n2. Internet of Things (IoT) applications: Flux can be used to collect and process data from various IoT devices, enabling the creation of smart and connected systems.\n\n3. Financial services: Flux's real-time capabilities make it well-suited for financial applications, such as high-frequency trading, fraud detection, and risk analysis.\n\n4. E-commerce and recommendation systems: Flux can be used to process and analyze user data in real-time, allowing for personalized recommendations and targeted advertising.\n\n5. Social media analytics: Flux's ability to handle streaming data makes it a good choice for analyzing social media feeds in real-time, enabling sentiment analysis, trend detection, and social network analys

### Reusing persisted chroma collection

In [50]:
# check if the existing collection has documents
channel_id = "@runonflux"
collection_name = f"openai_embeddings_{channel_id[1:]}"

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_collection(collection_name)
collection.count()

3117

In [51]:
# load from disk

openai_embeddings_function = OpenAIEmbeddings()

openai_client = Chroma(
    persist_directory="./chroma", 
    collection_name=collection_name, 
    embedding_function=openai_embeddings_function
)

In [52]:
query = "fud"
docs = openai_client.similarity_search(query, k=20)
# print results
for item in docs[:10]:
    print(f"- {item.page_content}")

- People love to Fud and create unneeded distractions
- I hope so too and we all hope so. But let's say that seeing your deleted message doesn't do much pleasure. And I certainly didn't take the liberty of making fud.
- lets hope for the best
- it's up to date mate 
- I think that now come dump. 
- chaos trouble violence disorder destruction confusion havocfracas commotion
- reddug akhtarg need to be more vigilant with fake admin tagging
- reddug could I do you ?
- Ill give it another go
- this is probably the only thing holding the project back


In [81]:
template= """
The context given are several messages retrieved from a chat about a blockchain project. Based on those messages try to answer the user question.
----------------
context: {context}
user question: {question}
"""

In [82]:
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(), 
    chain_type="stuff", 
    retriever=openai_client.as_retriever(search_type="mmr", search_kwargs={'fetch_k': 100}), 
    return_source_documents=True,
    verbose=True,
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["context", "question"],
        ),
    }
)

In [83]:
print(qa.combine_documents_chain.llm_chain.prompt)

input_variables=['context', 'question'] template='\nThe context given are several messages retrieved from a chat about a blockchain project. Based on those messages try to answer the user question.\n----------------\ncontext: {context}\nuser question: {question}\n'


In [84]:
query = "What makes this project unique?"
response = qa({"query":query})



> Entering new RetrievalQA chain...

> Finished chain.


In [85]:
response["result"]

'Based on the given context, it seems that the project mentioned (Flux) is being compared to other blockchain projects. One user mentions that the project is "probably the only thing holding the project back." Another user asks what features make Flux stand out compared to other chains.\n\nHowever, there is not enough information provided in the given messages to determine what specifically makes Flux unique. Further details or messages are needed to answer the user\'s question accurately.'